### 选取特征
最初是想直接尝试LSTM的，但是貌似对动态预测太高估了，效果很差。所以索性直接看成普通特征，用sklearn中的各种回归试试效果，在最后还试了一个两层的神经网络，所以这里划分特征是以前一天10分钟为间隔，总共144个时刻的`in_num`, `out_num`和各自的`paytype`。最后在拼接上一个描述这两天是否是工作日的向量，总共886维。标签是预测那一天中144个时刻的`in_num`或者`out_num`。

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
all_feature = np.zeros((81, 2, 23, 144 * 6 + 2))
all_label = np.zeros((81, 2, 23, 144))
all_min_max = {}
for istation in range(81):
    all_min_max['station-'+str(istation).zfill(2)] = {}
    tmp_min_max = {}
    station_file = pd.read_csv('./Metro_station/station-'+str(istation).zfill(2)+'.csv')
    tmp_min_max['in_num_min'] = station_file.iloc[:,2].min()
    tmp_min_max['in_num_max'] = station_file.iloc[:,2].max()
    tmp_min_max['in_paytype_0_min'] = station_file.iloc[:,3].min()
    tmp_min_max['in_paytype_0_max'] = station_file.iloc[:,3].max()
    tmp_min_max['in_paytype_1_min'] = station_file.iloc[:,4].min()
    tmp_min_max['in_paytype_1_max'] = station_file.iloc[:,4].max()
    tmp_min_max['in_paytype_2_min'] = station_file.iloc[:,5].min()
    tmp_min_max['in_paytype_2_max'] = station_file.iloc[:,5].max()
    tmp_min_max['in_paytype_3_min'] = station_file.iloc[:,6].min()
    tmp_min_max['in_paytype_3_max'] = station_file.iloc[:,6].max()
    tmp_min_max['out_num_min'] = station_file.iloc[:,7].min()
    tmp_min_max['out_num_max'] = station_file.iloc[:,7].max()
    tmp_min_max['out_paytype_0_min'] = station_file.iloc[:,8].min()
    tmp_min_max['out_paytype_0_max'] = station_file.iloc[:,8].max()
    tmp_min_max['out_paytype_1_min'] = station_file.iloc[:,9].min()
    tmp_min_max['out_paytype_1_max'] = station_file.iloc[:,9].max()
    tmp_min_max['out_paytype_2_min'] = station_file.iloc[:,10].min()
    tmp_min_max['out_paytype_2_max'] = station_file.iloc[:,10].max()
    tmp_min_max['out_paytype_3_min'] = station_file.iloc[:,11].min()
    tmp_min_max['out_paytype_3_max'] = station_file.iloc[:,11].max()
    if istation != 54:
        station_file.iloc[:,2:-1] = (station_file.iloc[:,2:-1] - station_file.iloc[:,2:-1].min()) / (station_file.iloc[:,2:-1].max() - station_file.iloc[:,2:-1].min())
    for jinout in range(2):
        if jinout == 0:
            all_min_max['station-'+str(istation).zfill(2)]['in'] = {}
            all_min_max['station-'+str(istation).zfill(2)]['in']['label_min'] = tmp_min_max['in_num_min']
            all_min_max['station-'+str(istation).zfill(2)]['in']['label_max'] = tmp_min_max['in_num_max']
            all_min_max['station-'+str(istation).zfill(2)]['in']['in_num_min'] = tmp_min_max['in_num_min']
            all_min_max['station-'+str(istation).zfill(2)]['in']['in_num_max'] = tmp_min_max['in_num_max']
            all_min_max['station-'+str(istation).zfill(2)]['in']['out_num_min'] = tmp_min_max['out_num_min']
            all_min_max['station-'+str(istation).zfill(2)]['in']['out_num_max'] = tmp_min_max['out_num_max']
            all_min_max['station-'+str(istation).zfill(2)]['in']['in_paytype_0_min'] = tmp_min_max['in_paytype_0_min']
            all_min_max['station-'+str(istation).zfill(2)]['in']['in_paytype_0_max'] = tmp_min_max['in_paytype_0_max']
            all_min_max['station-'+str(istation).zfill(2)]['in']['in_paytype_1_min'] = tmp_min_max['in_paytype_1_min']
            all_min_max['station-'+str(istation).zfill(2)]['in']['in_paytype_1_max'] = tmp_min_max['in_paytype_1_max']
            all_min_max['station-'+str(istation).zfill(2)]['in']['in_paytype_2_min'] = tmp_min_max['in_paytype_2_min']
            all_min_max['station-'+str(istation).zfill(2)]['in']['in_paytype_2_max'] = tmp_min_max['in_paytype_2_max']
            all_min_max['station-'+str(istation).zfill(2)]['in']['in_paytype_3_min'] = tmp_min_max['in_paytype_3_min']
            all_min_max['station-'+str(istation).zfill(2)]['in']['in_paytype_3_max'] = tmp_min_max['in_paytype_3_max']
        else:
            all_min_max['station-'+str(istation).zfill(2)]['out'] = {}
            all_min_max['station-'+str(istation).zfill(2)]['out']['label_min'] = tmp_min_max['out_num_min']
            all_min_max['station-'+str(istation).zfill(2)]['out']['label_max'] = tmp_min_max['out_num_max']
            all_min_max['station-'+str(istation).zfill(2)]['out']['in_num_min'] = tmp_min_max['in_num_min']
            all_min_max['station-'+str(istation).zfill(2)]['out']['in_num_max'] = tmp_min_max['in_num_max']
            all_min_max['station-'+str(istation).zfill(2)]['out']['out_num_min'] = tmp_min_max['out_num_min']
            all_min_max['station-'+str(istation).zfill(2)]['out']['out_num_max'] = tmp_min_max['out_num_max']
            all_min_max['station-'+str(istation).zfill(2)]['out']['out_paytype_0_min'] = tmp_min_max['out_paytype_0_min']
            all_min_max['station-'+str(istation).zfill(2)]['out']['out_paytype_0_max'] = tmp_min_max['out_paytype_0_max']
            all_min_max['station-'+str(istation).zfill(2)]['out']['out_paytype_1_min'] = tmp_min_max['out_paytype_1_min']
            all_min_max['station-'+str(istation).zfill(2)]['out']['out_paytype_1_max'] = tmp_min_max['out_paytype_1_max']
            all_min_max['station-'+str(istation).zfill(2)]['out']['out_paytype_2_min'] = tmp_min_max['out_paytype_2_min']
            all_min_max['station-'+str(istation).zfill(2)]['out']['out_paytype_2_max'] = tmp_min_max['out_paytype_2_max']
            all_min_max['station-'+str(istation).zfill(2)]['out']['out_paytype_3_min'] = tmp_min_max['out_paytype_3_min']
            all_min_max['station-'+str(istation).zfill(2)]['out']['out_paytype_3_max'] = tmp_min_max['out_paytype_3_max']
        for kday in range(23):
            if jinout == 0:
                station_feature = station_file.iloc[144*kday:144*(kday+2), [2,7,3,4,5,6,12]]
            else:
                station_feature = station_file.iloc[144*kday:144*(kday+2), [2,7,8,9,10,11,12]]
            for mseqlen in range(144, 288):
                if jinout == 0:
                    one_label = station_feature.iloc[144:,0].values
                else:
                    one_label = station_feature.iloc[144:,1].values
                one_feature = np.hstack((station_feature.iloc[:144,:-1].values.reshape(-1), station_feature.iloc[[1,-2], -1]))
                all_feature[istation, jinout, kday, :] = one_feature
                all_label[istation, jinout, kday, :] = one_label

### 划分训练集测试集
元旦那天比较特殊，剔除。按照每两天划分作为一个样本，总共有23个样本，其中测试样本根据工作日情况选取为`[22,23]`（都为工作日：00）、`[18,19]`（前一天工作日，后一天不是工作日：01）、`[20,21]`（前一天不是工作日，后一天工作日：10）和`[19,20]`（都不是工作日：11）。

In [ ]:
train_feature = all_feature[:,:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,19,21,22],:]
train_label = all_label[:,:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,19,21,22],:]
test_feature = all_feature[:,:,[16,17,18,20],:]
test_label = all_label[:,:,[16,17,18,20],:]

In [ ]:
np.save('train_feature.npy', train_feature)
np.save('train_label.npy', train_label)
np.save('test_feature.npy', test_feature)
np.save('test_label.npy', test_label)
np.save('all_min_max.npy', all_min_max)